In [ ]:
# 0) 최신 버전 설치
!pip -q install -U langchain langchain-openai langchain-community langsmith


In [ ]:

# 1) (선택) OpenAI 사용 시
import os, uuid
from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
LangSmith_KEY = os.getenv("LANGCHAIN_API_KEY")

# 2) LangSmith 연동 필수 환경변수
os.environ["LANGCHAIN_TRACING_V2"] = "true"      # 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"]   = "https://api.smith.langchain.com"  # 기본값
os.environ["LANGSMITH_PROJECT"]    = "llm_colab_ex_1"                 # 수업용 프로젝트명


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # 키 필요
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a concise assistant."),
    ("user", "{question}")
])

chain = prompt | llm | StrOutputParser()
print(chain.invoke({"question":"LangSmith를 왜 쓰나요?"}))


LangSmith는 주로 다음과 같은 이유로 사용됩니다:

1. **언어 모델 개발**: LangSmith는 언어 모델을 쉽게 개발하고 테스트할 수 있는 플랫폼을 제공합니다.
2. **협업 기능**: 팀원들과의 협업을 통해 모델을 개선하고 피드백을 받을 수 있습니다.
3. **데이터 관리**: 데이터셋을 효율적으로 관리하고, 다양한 형식으로 데이터를 처리할 수 있습니다.
4. **성능 분석**: 모델의 성능을 분석하고 최적화할 수 있는 도구를 제공합니다.
5. **사용자 친화성**: 직관적인 인터페이스로 사용자가 쉽게 접근할 수 있습니다.

이러한 기능들 덕분에 LangSmith는 언어 처리 프로젝트에 유용한 도구로 자리잡고 있습니다.


In [ ]:
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from datetime import datetime

# === 1. 도구 정의 ===
@tool
def multiply(a: int, b: int) -> int:
    """두 수를 곱한다."""
    return a * b

@tool
def now_kst() -> str:
    """현재 UTC 시간을 반환한다 (KST 아님 예제용)."""
    return datetime.utcnow().strftime("UTC %Y-%m-%d %H:%M:%S")

tools = [multiply, now_kst]

# === 2. LLM ===
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# === 3. 프롬프트 (⚠️ agent_scratchpad 반드시 포함해야 함) ===
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 툴-사용 에이전트야. 계산은 multiply를, 시간은 now_kst를 사용해야 한다."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")   # ← 필수
])

# === 4. 에이전트 생성 ===
agent = create_tool_calling_agent(llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# === 5. 실행 ===
config = {
    "tags": ["demo", "agent", "tools", "colab"],
    "metadata": {"lesson":"ls-tracing-01"},
    "run_name": "agent_tools_demo"
}

print(executor.invoke(
    {"input": "23과 7을 곱하고, 현재 시간을 알려줘."},
    config=config
))




> Entering new agent_tools_demo chain...

Invoking: `multiply` with `{'a': 23, 'b': 7}`


161
Invoking: `now_kst` with `{}`


UTC 2025-08-28 03:07:01

/tmp/ipython-input-2488277466.py:16: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().strftime("UTC %Y-%m-%d %H:%M:%S")


23과 7을 곱한 결과는 161입니다. 현재 시간은 2025년 8월 28일 03:07:01 UTC입니다.

> Finished chain.
{'input': '23과 7을 곱하고, 현재 시간을 알려줘.', 'output': '23과 7을 곱한 결과는 161입니다. 현재 시간은 2025년 8월 28일 03:07:01 UTC입니다.'}


In [ ]:
from langsmith import traceable
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

# === 1) 간단한 로컬 문서 코퍼스 ===
CORPUS = [
    {"title":"LangSmith 소개", "text":"LangSmith는 LLM 앱을 관찰, 평가, 디버깅할 수 있는 플랫폼입니다."},
    {"title":"LangChain 소개", "text":"LangChain은 LLM 기반 앱을 체인으로 구성해 쉽게 개발/운영하도록 돕는 프레임워크입니다."},
    {"title":"RAG 개요", "text":"RAG는 검색과 생성 결합 기법으로, 외부 지식을 검색해 답변의 정확성을 높입니다."},
]

# === 2) 추출기(retriever) & 요약기(summarizer)를 traceable로 감싸기 ===
@traceable(name="keyword_retriever")
def keyword_retrieve(query: str, topk: int = 2):
    q = query.lower()
    scored = []
    for doc in CORPUS:
        score = sum(1 for tok in q.split() if tok in doc["text"].lower())
        scored.append((score, doc))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [d for s, d in scored[:topk] if s > 0] or [CORPUS[0]]

@traceable(name="summarize_docs")
def summarize_docs(docs):
    joined = "\n".join([f"- {d['title']}: {d['text']}" for d in docs])
    return joined

# === 3) 체인 구성: retrieve -> summarize -> prompt -> LLM -> parser ===
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_template(
    "질문: {question}\n"
    "관련 문서:\n{context}\n"
    "문서 근거를 바탕으로 한 문단으로 간결히 답해줘."
)

def rag_pipeline(question: str):
    docs = keyword_retrieve(question)
    ctx = summarize_docs(docs)
    chain = (
        {"question": RunnablePassthrough(), "context": lambda q: ctx}
        | prompt
        | llm
        | StrOutputParser()
    )
    # tags/metadata/run_name을 붙여 LangSmith에서 찾기 쉽게!
    return chain.invoke(question, config={
        "tags": ["demo", "rag", "batch", "colab"],
        "metadata": {"lesson":"ls-tracing-02"},
        "run_name": "mini_rag_pipeline"
    })

# === 4) 단건 실행 ===
print("단건 실행:\n", rag_pipeline("LangSmith와 RAG의 관계는?"))

# === 5) 배치 실행 (여러 질문을 한꺼번에) ===
questions = [
    "LangChain은 무엇이며 어떤 역할을 하나요?",
    "RAG가 답변 품질에 기여하는 방식은?",
    "LangSmith를 활용하면 개발자가 얻는 이점은?"
]

# .batch를 쓰려면 chain을 Runnable로 구성해야 하므로, 위 람다를 조금 변경한 버전:
def build_rag_chain():
    # Retrieval과 Summarization은 traceable 함수로 개별 런이 생김
    def _ctx_fn(q):
        return summarize_docs(keyword_retrieve(q))
    chain = (
        {"question": RunnablePassthrough(), "context": _ctx_fn}
        | prompt
        | llm
        | StrOutputParser()
    )
    return chain

rag_chain = build_rag_chain()

batch_outputs = rag_chain.batch(
    questions,
    config={"tags":["demo","rag","batch"], "metadata":{"lesson":"ls-tracing-02"}, "run_name":"mini_rag_batch"},
    return_exceptions=True  # 에러도 LangSmith에 기록됩니다
)

print("\n배치 실행 결과:")
for q, ans in zip(questions, batch_outputs):
    print(f"- Q: {q}\n  A: {ans}\n")


단건 실행:
 LangSmith는 LLM(대형 언어 모델) 앱을 관찰, 평가, 디버깅할 수 있는 플랫폼으로, RAG(정보 검색 기반 생성)와의 관계는 RAG가 LLM의 성능을 향상시키기 위해 외부 정보를 검색하고 활용하는 방식에 있습니다. LangSmith는 이러한 LLM 앱의 작동을 분석하고 최적화하는 도구를 제공함으로써 RAG의 효과적인 구현을 지원할 수 있습니다.

배치 실행 결과:
- Q: LangChain은 무엇이며 어떤 역할을 하나요?
  A: LangChain은 LLM(대형 언어 모델) 기반 애플리케이션을 체인 형태로 구성하여 쉽게 개발하고 운영할 수 있도록 돕는 프레임워크입니다. 이를 통해 개발자들은 복잡한 언어 모델을 효과적으로 활용하여 다양한 애플리케이션을 구축할 수 있습니다.

- Q: RAG가 답변 품질에 기여하는 방식은?
  A: RAG(검색-생성 결합 기법)는 외부 지식을 검색하여 답변의 정확성을 높이는 방식으로 답변 품질에 기여합니다. 이를 통해 사용자는 보다 신뢰할 수 있는 정보를 제공받을 수 있으며, 생성된 답변이 최신의 관련 데이터를 반영하게 됩니다. 이러한 과정은 정보의 정확성과 신뢰성을 동시에 향상시켜, 전반적인 답변 품질을 개선하는 데 중요한 역할을 합니다.

- Q: LangSmith를 활용하면 개발자가 얻는 이점은?
  A: LangSmith를 활용하면 개발자는 LLM 앱의 성능을 효과적으로 관찰하고 평가할 수 있으며, 디버깅 과정에서 발생하는 문제를 신속하게 해결할 수 있는 이점을 얻습니다. 이를 통해 개발자는 앱의 품질을 향상시키고, 사용자 경험을 개선할 수 있습니다.

